In [3]:
# %serialconnect to --port="/dev/tty.usbserial-02EDKZTP" --baud=115200

%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200

[leftinbuffer] ['>']
[leftinbuffer] ['MicroPython v1.15-222-g8edc3aacd-dirty on 2021-06-16; ESP32 module with ESP32']
[leftinbuffer] ['Type "help()" for more information.']
[leftinbuffer] ['>>> ']
Connecting to --port=/dev/tty.usbserial-02U1W54L --baud=115200 
MicroPython v1.15-222-g8edc3aacd-dirty on 2021-06-16; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [4]:
from ulab import numpy as np
import machine
from machine import Pin, PWM

In [5]:
import utime

led = Pin(5, Pin.OUT)

for d in range(1, 8):
    for i in range(4):
        led.on()
        utime.sleep(0.2/d)
        led.off()
        utime.sleep(0.2/d)

In [11]:
from lib.scheduling import LedFlasher

flasher = LedFlasher(0, 7, led)
flasher.start()

In [6]:
flasher.stop()

In [25]:
import urandom

def synth_x(f, Ns, noise_power=0.5, fs=250):
    """
    generate a synthetic signal vector
    
    args:
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    """
    t = np.arange(0, Ns/fs, 1/fs)
    return np.sin(t*2*np.pi*f)

def synth_X(f, Nc, Ns, noise_power=0.5, fs=200, f_std=0.02, noise_std=0.2):
    """
    Generate a matrix of several variations of the same target signal. This is used
    to simulate the measurement of a common signal over multiple EEG channels 
    that have different SNR characteristics.
    
    args:
    f [float]: target frequency of synthetic signal (Hz)
    Nc [int]: number of channels
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    fs [float]: sampling frequency (Hz)
    f_std [float]: standard dev. of freq. in generated signal across channels to simulate interference from other frequency components over different channels
    noise_std [float]: standard dev. of noise across channels
    """
    X = []
    for i in range(Nc): # simulate noisy sinusoids with varying SNR across Nc channels
        f_i = f*(1+urandom.random()*0.1)
        x = synth_x(f_i, Ns, noise_power=0)

        X.append(x)
        
    return np.array(X)

In [13]:
import urandom

A=np.array([[ -26362.3397184 ,  -26184.6722884 ,  -26220.54833099,
         -26136.40472891],
       [-700856.71802712, -696101.61106368, -697056.58743602,
        -694819.37609336],
       [ 473937.13966951,  470722.80110627,  471368.87718473,
         469855.05264848],
       [ 253278.3958703 ,  251559.98998877,  251904.76024221,
         251097.24412795]])


# print(max_eig(A, power_iterations=100))

iterations = 100

b_k = np.array([urandom.random() for i in range(len(A))])

for _ in range(iterations):
    b_k1 = np.dot(A, b_k)
    b_k1_norm = np.linalg.norm(b_k1)
    # re normalize the vector
    b_k = b_k1 / b_k1_norm
    
b_k_vec = b_k.reshape((len(b_k), 1))
lam = np.dot(np.dot(b_k_vec.transpose(), A), b_k)/(np.dot(b_k_vec.transpose(), b_k_vec))
    

print(b_k_vec, lam)

array([[-0.03202198],
       [-0.5123517],
       [0.8539195],
       [-0.08539195]], dtype=float32) array([[166237.3]], dtype=float32)


In [22]:
from lib.computation import solve_eig_qr

lam, V = solve_eig_qr(np.dot(A, A.transpose()), 2)
print(lam[0], V[:, 0])

3.090257e+12 array([-0.0298378, -0.7932273, 0.5364015, 0.2866594], dtype=float32)


In [49]:
from lib.computation import max_eig

X = X_test

Cxx = np.dot(X, X.transpose())
Cyy = np.dot(Y, Y.transpose()) 
Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

M1 = np.dot(np.linalg.inv(Cxx), Cxy) # intermediate result
M2 = np.dot(np.linalg.inv(Cyy), Cyx)

lam, _ = max_eig(np.dot(M1, M2))

print(np.dot(M1, M2))

array([[1387.354, 1364.06, 1389.151, 1389.836],
       [-313.0255, -306.9955, -313.5925, -313.8571],
       [644.5729, 633.5605, 645.446, 645.7906],
       [-1278.465, -1257.528, -1280.01, -1280.566]], dtype=float32)


In [26]:
from lib.decoding import harmonic_reference, CCA
from lib.computation import solve_gen_eig_prob
from ulab import numpy as np

gc.collect()

f0 = 7
stim_freqs = [7, 10, 12]
fs = 200

X_test = synth_X(10, 4, 10)

cca = CCA(stim_freqs, fs)

print(cca.compute_corr(X_test))


{12: 21.09067, 10: 21.07713, 7: 21.36454}


In [34]:
import gc
import micropython

gc.collect()

# X = synth_X(10, 4, 1000)
print(micropython.mem_info())

stack: 736 out of 15360
GC: total: 111168, used: 19168, free: 92000
 No. of 1-blocks: 57, 2-blocks: 25, max blk sz: 500, max free sz: 2140
None


In [ ]:
%ls

In [1]:
%ls --recursive lib

No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [ ]:
%sendtofile --source lib/mpy/computation.mpy /lib/computation.mpy 

In [238]:
%sendtofile lib/decoding.py --source lib/decoding.py

Sent 55 lines (1976 bytes) to lib/decoding.py.


In [19]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.15-222-g8edc3aacd-dirty on 2021-06-16; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.15-222-g8edc3aacd-dirty on 2021-06-16; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [20]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--load] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%ls [--recurse] [dirname]
    list files on the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute] [--source [SOURCE]] [--quiet]
                   [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over US